In [9]:
import cv2
import numpy as np
import yaml

In [10]:
max_matches=100
path='cam1.yaml'

In [11]:
# load camera intrinsics
with open(path, 'r') as f:
    data = yaml.safe_load(f)
    K = np.array(data['camera_matrix']).reshape(3, 3)
    dist_coeffs = np.array(data['dist_coeff']).reshape(-1, 1)
# Load images
img1 = cv2.imread('left_image.jpg')
img2 = cv2.imread('right_image.jpg')

In [12]:
print("Camera Matrix:\n", K)
print("Distortion Coefficients:", dist_coeffs.T)

Camera Matrix:
 [[901.44204508   0.         627.54480678]
 [  0.         904.66861291 379.96878104]
 [  0.           0.           1.        ]]
Distortion Coefficients: [[ 2.90064547e-01 -1.34251639e+00 -1.08082547e-03 -1.91922978e-03
   2.35356175e+00]]


In [13]:
# find and match features
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

# Get all matches first
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

# Calculate Fundamental matrix and find inliers
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# Select inliers
inliers_mask = mask.ravel() == 1
pts1_inliers = pts1[inliers_mask]
pts2_inliers = pts2[inliers_mask]
matches_inliers = [m for i, m in enumerate(matches) if inliers_mask[i]]

# Limit the number of matches if necessary
if len(matches_inliers) > max_matches:
    good_matches = matches_inliers[:max_matches]
    pts1_inliers = pts1_inliers[:max_matches]
    pts2_inliers = pts2_inliers[:max_matches]

# return kp1, kp2, pts1_inliers, pts2_inliers, matches_inliers, len(inliers_mask), np.sum(inliers_mask)

In [4]:
print(f"Total matches: {total_matches}")
print(f"Inliers count: {inliers_count}")



Total matches: 161
Inliers count: 111


In [14]:
# Visualize matched points
img3 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=2)
cv2.imshow("Matched Features", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
def calculate_3D_point(img1_points, img2_points, K, baseline):
    R = np.eye(3)
    T = np.array([[baseline, 0, 0]]).T
    
    P1 = np.hstack((K, np.zeros((3, 1))))
    P2 = K @ np.hstack((R, T))
    
    points_4D_hom = cv2.triangulatePoints(P1, P2, img1_points.T, img2_points.T)
    
    # Check if triangulation was successful
    if np.any(points_4D_hom[3] == 0):
        valid_mask = points_4D_hom[3] != 0
        points_4D_hom = points_4D_hom[:, valid_mask]
        
    points_3D = points_4D_hom[:3] / points_4D_hom[3]
    return points_3D.T

In [11]:
baseline = 13.1  # cm
points_3D = calculate_3D_point(pts1, pts2, K, baseline)
distances = np.linalg.norm(points_3D, axis=1)

print(f"Distance to object: {np.mean(distances):.2f} cm")
print(f"Min distance: {np.min(distances):.2f} cm")
print(f"Max distance: {np.max(distances):.2f} cm")




Distance to object: 40.38 cm
Min distance: 38.91 cm
Max distance: 45.66 cm
